# duckduckgo

In [23]:
import torch
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
#import requests
from duckduckgo_search import DDGS

def image_to_text(image_path):
    # Load the BLIP model and processor
    device = "cuda" if torch.cuda.is_available() else "cpu"
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    # Load and preprocess the image
    image = Image.open(image_path)

    # Preprocess the image and generate caption
    inputs = processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(**inputs)

    # Decode the generated caption
    caption = processor.decode(generated_ids[0], skip_special_tokens=True)
    print(caption)
    return caption

# Define the function for web search using ContextualWebSearch API
""" 
Args:
keywords: keywords for query.
# https://duckduckgo.com/duckduckgo-help-pages/settings/params/
region: wt-wt, us-en, uk-en, ru-ru, etc. Defaults to "wt-wt".
safesearch: on, moderate, off. Defaults to "moderate".
timelimit: d, w, m, y. Defaults to None.
backend: api, html, lite. Defaults to api.
api html collect data from https://duckduckgo.com, collect data from https://html.duckduckgo.com,
lite collect data from https://lite.duckduckgo.com.
max_results: max number of results. If None, returns results only from the first response. Defaults to None.
"""
def perform_web_search(query):

    results=DDGS().text(
        keywords=query,
        region='in-en',
        safesearch='off',
        timelimit=None,
        max_results=50
    )
    results_df=pd.DataFrame(results)
    return results_df

if __name__ == "__main__":
    # Example usage
    image_path ="horse.jpeg"  # Replace with your image path
    #text_prompt = ["red hoodie","a bright red hoodie"]
    text_prompt="search for this types"
    search_query=image_to_text(image_path) + text_prompt

    search_results=perform_web_search(search_query)
    search_results.to_csv('results.csv',index=False)
    #print(f"Found {len(search_results)} search results")

c:\Users\pushp_vnmc973\Documents\ML\PS1\.venv\Lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a red hoodie sweatshirt with a white hoodie
